# Lab: Build a TOR-Like Server Node

In this lab, you will build server nodes that work together to create a TOR-like network. Each node will listen on a specific port, decrypt incoming packets, and forward them to the next node or send the actual request if it is the last node. The nodes will ensure secure and anonymous communication by encrypting and decrypting the data at each step.

### Objectives:

1. **Listening on a Specific Port**: Each node will listen on a designated port for incoming connections.

2. **Receiving and Decrypting Packets**: When a node receives a connection, it will receive the packet and decrypt its layer of encryption.

3. **Forwarding to the Next Node**: 
    - If the decrypted packet contains an IP address and port, the node will forward the remaining encrypted packet to the next node in the circuit.
    - The packet forwarded will still be encrypted (it will be the second layer of encryption).

4. **Sending the Actual Request**:
    - If the node is the last in the circuit, upon decryption, it will reveal the actual HTTP request.
    - The node will send the HTTP request to the target server and obtain the response.

5. **Returning the Response**:
    - The node will return the response to the parent node, encrypting it with the parent node's public key to maintain the security and anonymity of the communication. The response must follow the circuit until it gets to the client.

### Steps:

1. **Listening on a Specific Port**:
    - Set up each node to listen on a designated port for incoming connections.

2. **Receiving and Decrypting Packets**:
    - When a node receives a packet, it will decrypt its layer using its private key.

3. **Forwarding to the Next Node**:
    - If the decrypted packet contains an IP address and port, the node will forward the remaining encrypted packet to the next node in the circuit.
    - Ensure the packet remains encrypted for the next node.

4. **Sending the Actual Request**:
    - If the node is the last in the circuit, it will decrypt the packet to reveal the HTTP request.
    - Send the HTTP request to the target server and obtain the response.

5. **Returning the Response**:
    - Encrypt the response with the parent node's public key.
    - Send the encrypted response back through the circuit to the client.

### Tips:

Watchout with the lenght of the packets. Most encryption errors could be due this, so you'll maybe have to send and handle chunks. Every time the packet is encrypted, it's size will change

In [ ]:
#Note: This is how I did it. It's just one way, there are multiple ways on doing this.

import socket
import threading
import os
import ssl
import base64
from Crypto.PublicKey import RSA
from time import sleep
from Crypto.Cipher import PKCS1_OAEP

def generate_rsa_key_pair():
    # Generate RSA key pair
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()

    # Store keys in variables
    private_key_var = RSA.import_key(private_key)
    public_key_var = RSA.import_key(public_key)
    return private_key_var, public_key_var
# Function to encrypt a message using the public key
def encrypt_message(message, public_key):
    # Encrypt message
    return encoded_encrypted_message
# Function to decrypt a message using the private key
def decrypt_message(encrypted_message, private_key):
    return decrypted_message


class Node:
    PORT_START = 5000
    def __init__(self, id, prev_node=None, next_node=None):
        self.id = id
        self.port = self.PORT_START + id
        self.prev_node:Node = prev_node
        self.next_node:Node = next_node
        self.private_key, self.public_key = generate_rsa_key_pair()


    def handle_client(self, conn:socket.socket, addr):
        try:
            sleep(1)  # Wait for the client to send data
            conn.settimeout(1.0)  # Set a timeout for receiving data
            tf = True
            while True:
                try:
                    data = conn.recv(4098)
                    if not data:
                        break
                    #process data
                except socket.timeout:
                    break
                except Exception as e:
                    print(f"Node {self.id} Error during receiving: {e}")
                    break
            
            if nodeNum > 0:
                pass
               #Send it to the next node
            else:
                #Decrypt the data

                #Create a SSL Socket
                dest_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
                
                #Get the final host from the HTTP Header
                address = self.extract_host(decrypted_data)
                
                with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as dest_socket:
                    with context.wrap_socket(dest_socket, server_hostname=address) as ssock:
                        #Connect to the final host
                        ssock.connect((address, 443))
                        #Send the decrypted data to the final host
                        ssock.sendall(decrypted_data)
                        #Receive the response from the final host
                        response = ssock.recv(4096)
                        #Encrypt the response

                        #Send the encrypted response to the previous node
                        self.send_data(encrypted_response, conn)
                        

        except Exception as e:
            print(f"Node {self.id} Error: {e}")
        finally:
            conn.close()
    #This function extracts the host from the HTTP Header
    def extract_host(self,request_bytes):
        request_str = request_bytes.decode('utf-8')  # Decode the bytes to a string
        lines = request_str.split('\r\n')  # Split the request into lines
        for line in lines:
            if line.startswith('Host: '):
                host = line.split(' ')[1]  # Extract the host part
                return host
        return None
    def send_data(self, data, s=None):
        #Send data to the node. In my implementation, s is a socket, the same from which the data was received. If it's sending to a new socket, s should be None and a new socket should be created.
        pass

    def start(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(('127.0.0.1', self.port))
            s.listen()
            print(f"Node {self.id} listening on port {self.port}")
            while True:
                conn, addr = s.accept()
                client_thread = threading.Thread(target=self.handle_client, args=(conn, addr))
                client_thread.start()